In [1]:
!pip install --pre deepchem

     |████████████████████████████████| 608 kB 27.7 MB/s 
     |████████████████████████████████| 20.6 MB 1.4 MB/s 


In [2]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y
!pip install --pre deepchem

--2021-12-28 02:08:25--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   214MB/s    in 0.4s    

2021-12-28 02:08:26 (214 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6

In [4]:
# data plotting: before split
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import deepchem as dc

In [8]:
df = pd.read_csv('chembl_bace_bioactivity_data_raw.csv')
df1 = df[['molecule_chembl_id', 'canonical_smiles', 'pchembl_value']]

In [9]:
df2 = df1[df1.canonical_smiles.notna()]
df3 = df2[df2.pchembl_value.notna()]

In [13]:
df3.to_csv('chembl_bace_bioactivity_data_raw_na_removed.csv', index=False)

In [14]:
import deepchem as dc
import pandas as pd
import os

# pandas read file
original_data = pd.read_csv('chembl_bace_bioactivity_data_raw_na_removed.csv')
#original_data_sub = original_data[['mol', 'CID', 'Class', 'pIC50']]
# deepchem read file
input_data='chembl_bace_bioactivity_data_raw_na_removed.csv'
tasks=['pchembl_value']
featurizer=dc.feat.CircularFingerprint(size=1024)
loader = dc.data.CSVLoader(tasks=tasks, feature_field='canonical_smiles',featurizer=featurizer)
dataset=loader.create_dataset(input_data)
# splitters
randomSplitter = dc.splits.RandomSplitter()
#scaffoldsplitter = dc.splits.ScaffoldSplitter()
#butinaSplitter = dc.splits.ButinaSplitter()
splitters = [randomSplitter]
for splitter in splitters:
  # split data
  train_dataset, test_dataset = splitter.train_test_split(dataset)
  # train data get smiles
  train_dataset_df = train_dataset.to_dataframe()
  train_dataset_df_sub = train_dataset_df[['ids']]
  train_dataset_df_sub.columns = ['smiles']
  # train data merge
  merged_train_data = pd.merge(train_dataset_df_sub, original_data, how='inner', left_on='smiles', right_on='canonical_smiles', left_index=False, right_index=False)
  merged_train_data_1 = merged_train_data[merged_train_data.canonical_smiles.notna()]
  merged_train_data_2 = merged_train_data_1[merged_train_data_1.pchembl_value.notna()]
  
  merged_train_data.to_csv('chembl_bace1_train_'+ str(splitter) + '.csv', index=False)
  # test data get smiles
  test_dataset_df = test_dataset.to_dataframe()
  test_dataset_df_sub = test_dataset_df[['ids']]
  test_dataset_df_sub.columns = ['smiles']
  # test data merge
  merged_test_data = pd.merge(test_dataset_df_sub, original_data, how='inner', left_on='smiles', right_on='canonical_smiles', left_index=False, right_index=False)
  merged_test_data_1 = merged_test_data[merged_test_data.canonical_smiles.notna()]
  merged_test_data_2 = merged_test_data_1[merged_test_data_1.pchembl_value.notna()]
  
  merged_test_data.to_csv('chembl_bace1_test_'+ str(splitter) + '.csv', index=False)

In [15]:
import numpy as np
import deepchem as dc
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [16]:
def create_dataset(file_name, mol_smi, pIC50):
  data = pd.read_csv(file_name)
  data_x = data[mol_smi]
  mols = [Chem.MolFromSmiles(smi) for smi in data_x]
  morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in mols]
  morgan_fps_array = np.asarray(morgan_fps, dtype=float)
  maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in mols]
  maccs_fps_array = np.asarray(maccs_fps, dtype=float)
  x = np.concatenate([morgan_fps_array, maccs_fps_array],axis=1)
  y = data[pIC50]
  y = np.asarray(y, dtype=float).reshape(-1, 1)
  dataset = dc.data.NumpyDataset(X=x, y=y)
  return dataset

In [17]:
def keras_reg(train_fin, test_fin, mol_smi,pIC50):
    train_dataset = create_dataset(train_fin, mol_smi, pIC50)
    test_dataset = create_dataset(test_fin, mol_smi, pIC50)
    
    keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1)])
    model = dc.models.KerasModel(keras_model, dc.models.losses.L2Loss())
 
    res_list = []
    for epoch in [10, 20, 30]:

        model.fit(train_dataset, nb_epoch=epoch)

        train_predict = model.predict(train_dataset)    
        train_predict_list = train_predict.reshape(-1).tolist()
        train_y_list = train_dataset.y.tolist()

        train_mae = mean_absolute_error(train_predict_list, train_y_list)
        train_mse = mean_squared_error(train_predict_list, train_y_list)
        train_mape = mean_absolute_percentage_error(train_predict_list, train_y_list)
        train_r2 = r2_score(train_predict_list, train_y_list)

        test_predict = model.predict(test_dataset)
        test_predict_list = test_predict.reshape(-1).tolist()
        test_y_list = test_dataset.y.tolist()   

        test_mae = mean_absolute_error(test_predict_list, test_y_list)
        test_mse = mean_squared_error(test_predict_list, test_y_list)
        test_mape = mean_absolute_percentage_error(test_predict_list, test_y_list)
        test_r2 = r2_score(test_predict_list, test_y_list)

        keras_perf = {'mae':[train_mae, test_mae],
                'mse':[train_mse, test_mse],
                'mape':[train_mape, test_mape],
                'r2':[train_r2, test_r2]}
        res_list.append(pd.DataFrame.from_dict(keras_perf))
    final_res = pd.concat(res_list, keys = [10, 20, 30])
        
    return final_res

In [18]:
! ls

chembl_bace1_test_RandomSplitter.csv
chembl_bace1_train_RandomSplitter.csv
chembl_bace_bioactivity_data_raw.csv
chembl_bace_bioactivity_data_raw_na_removed.csv
Miniconda3-py37_4.8.2-Linux-x86_64.sh
sample_data


In [19]:
chembl_bace1 = keras_reg('chembl_bace1_train_RandomSplitter.csv', 'chembl_bace1_test_RandomSplitter.csv', 'canonical_smiles', 'pchembl_value')

In [20]:
chembl_bace1

mae       mse      mape        r2
10 0  0.420848  0.342013  0.062113  0.679073
   1  0.473097  0.419407  0.071427  0.609035
20 0  0.419574  0.329258  0.063068  0.685971
   1  0.483160  0.426876  0.074482  0.594980
30 0  0.458882  0.388396  0.064282  0.682715
   1  0.521871  0.499846  0.073945  0.595417